In [ ]:
%run 'onedrive/MSc Project/TAP-MD/src/run_perturb.py' \
--image_path 'onedrive/MSc Project/KITTI/utils/kitti_eigen_test_image0.txt' \
--n_height 192 \
--n_width 640 \
--n_channel 3 \
--output_norm 0.02 \
--n_step 250 \
--learning_rates 4.0 1.0 \
--learning_schedule 400 \
--depth_method monodepth \
--depth_transform_func multiply \
--depth_transform_value 1.10 \
--mask_constraint none \
--checkpoint_path perturbations/monodepth/all_mult110_norm002_lr4e0_1e0_plain \
--depth_model_restore_path0 local/modelweights/plain/encoder-60.pth \
--depth_model_restore_path1 local/modelweights/plain/decoder-60.pth \
--device gpu

In [ ]:
%run 'onedrive/MSc Project/TAP-MD/src/run_perturb.py' \
--image_path 'onedrive/MSc Project/KITTI/utils/kitti_eigen_test_image0.txt' \
--n_height 192 \
--n_width 640 \
--n_channel 3 \
--output_norm 0.02 \
--n_step 250 \
--learning_rates 4.0 1.0 \
--learning_schedule 400 \
--depth_method monodepth \
--depth_transform_func multiply \
--depth_transform_value 1.10 \
--mask_constraint none \
--checkpoint_path perturbations/monodepth/all_mult110_norm002_lr4e0_1e0_adv \
--depth_model_restore_path0 local/modelweights/adv/encoder-100.pth \
--depth_model_restore_path1 local/modelweights/adv/decoder-100.pth \
--device gpu

In [ ]:
%run 'onedrive/MSc Project/TAP-MD/src/run_perturb.py' \
--image_path 'onedrive/MSc Project/KITTI/utils/kitti_eigen_test_image0.txt' \
--n_height 192 \
--n_width 640 \
--n_channel 3 \
--output_norm 0.02 \
--n_step 250 \
--learning_rates 4.0 1.0 \
--learning_schedule 400 \
--depth_method monodepth \
--depth_transform_func multiply \
--depth_transform_value 1.10 \
--mask_constraint none \
--checkpoint_path perturbations/monodepth/all_mult110_norm002_lr4e0_1e0_mixed \
--depth_model_restore_path0 local/modelweights/half/encoder-100.pth \
--depth_model_restore_path1 local/modelweights/half/decoder-100.pth \
--device gpu

In [ ]:
!git clone https://github.com/alexklwong/targeted-adversarial-perturbations-monocular-depth.git
!mv targeted-adversarial-perturbations-monocular-depth/* .

In [ ]:
import os, time
import numpy as np
import torch

import sys
sys.path.append('external_src/monodepth/src')
import datasets, data_utils
import global_constants as settings
from log_utils import log
from monodepth_model import MonodepthModel

In [ ]:
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred)**2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

In [ ]:
#Load Model
model = MonodepthModel(
        encoder_type='resnet18',
        decoder_type='up',
        activation_func='elu',
        n_pyramid=4,
        scale_factor=0.30,
        device='cuda')


loaded_dict_encoder = torch.load('onedrive/MSc Project/monodepth_models/resnet18advmphalf/encoder-100.pth', map_location=model.device)
loaded_dict_decoder = torch.load('onedrive/MSc Project/monodepth_models/resnet18advmphalf/decoder-100.pth', map_location=model.device)
model.encoder.load_state_dict(loaded_dict_encoder['model_state_dict'])
model.decoder.load_state_dict(loaded_dict_decoder['model_state_dict'])


In [ ]:
#Read test images
image0_paths = data_utils.read_paths('onedrive/MSc Project/KITTI/utils/kitti_eigen_test_image0.txt')
image1_paths = data_utils.read_paths('onedrive/MSc Project/KITTI/utils/kitti_eigen_test_image1.txt')
camera_paths = data_utils.read_paths('onedrive/MSc Project/KITTI/utils/kitti_eigen_test_camera.txt')
n_sample = len(image0_paths)


for i in range(len(image0_paths)):
    image0_paths[i] = "local/trainingDataSet/" + image0_paths[i]
    image1_paths[i] = "local/trainingDataSet/" + image1_paths[i]
    camera_paths[i] = "local/trainingDataSet/" + camera_paths[i]


dataloader = torch.utils.data.DataLoader(
    datasets.ImagePairCameraDataset(
        image0_paths,
        image1_paths,
        camera_paths,
        shape=(192, 640)),
    batch_size=1,
    shuffle=False,
    num_workers=1,
    drop_last=False)

In [ ]:
#Test plain model at 60th epoch.
#FGSM
import cv2

modelName = 'plain'
locNames = ['plain', 'adv', 'half']

epochNums = [60, 100, 100]

n_sample = len(image0_paths)

gt_depths = np.load('gt_depths.npy')

min_depth = 0
max_depth = 80

#Load Model
model = MonodepthModel(
        encoder_type='resnet18',
        decoder_type='up',
        activation_func='elu',
        n_pyramid=4,
        scale_factor=0.30,
        device='cuda')

for k in range(3):
    
    modelName = locNames[k]
    epochNum = epochNums[k]
    
    loaded_dict_encoder = torch.load("local/modelweights/"+ modelName +"/encoder-"+ str(epochNum) +".pth", map_location=model.device)
    loaded_dict_decoder = torch.load("local/modelweights/"+ modelName +"/decoder-"+ str(epochNum) +".pth", map_location=model.device)
    model.encoder.load_state_dict(loaded_dict_encoder['model_state_dict'])
    model.decoder.load_state_dict(loaded_dict_decoder['model_state_dict'])

    rms     = np.zeros((2, n_sample), np.float32)
    log_rms = np.zeros((2, n_sample), np.float32)
    abs_rel = np.zeros((2, n_sample), np.float32)
    sq_rel  = np.zeros((2, n_sample), np.float32)
    a1      = np.zeros((2, n_sample), np.float32)
    a2      = np.zeros((2, n_sample), np.float32)
    a3      = np.zeros((2, n_sample), np.float32)

    i = 0

    parameters = model.parameters()
    optimizer = torch.optim.SGD(parameters, lr=0.01)

    for image0, image1, camera in dataloader:
        print("\r" + str(i), end="")
        image0 = image0.cuda()
        image1 = image1.cuda()
        camera = camera.cuda()

        optimizer.zero_grad()
#         image0.requires_grad = True

        pred_depth = model.forward(image0, camera)
        pred_depth_orig = np.squeeze(pred_depth[2].detach().cpu().numpy())


#         #FGSM Vanilla Attack##################     
#         loss = model.compute_loss(image0, image1,
#                 w_color=settings.W_COLOR,
#                 w_ssim=settings.W_SSIM,
#                 w_smoothness=settings.W_SMOOTHNESS,
#                 w_left_right=settings.W_LEFT_RIGHT)

#         loss.backward()


#         #Get gradient of image
#         img_grad = image0.grad.data
#         #Sign
#         sign_img_grad = img_grad.sign()
#         epsilon = 8/255.
#         #pertubation
#         pertubation = epsilon*sign_img_grad
#         pert_img = image0 + pertubation
#         #Clip
#         pert_img = torch.clamp(pert_img, 0, 1)
#         ######################################

        #PGD Attack##################
        epsilon = 8/255.
        alpha = 2/255.

        delta = torch.zeros_like(image0).uniform_(-epsilon, epsilon).cuda()
        advImage = image0 + delta
        for k in range(7):

            delta.requires_grad = True
            advImage = image0 + delta
            model.forward(advImage, camera)
            loss = model.compute_loss(advImage, image1,
                    w_color=settings.W_COLOR,
                    w_ssim=settings.W_SSIM,
                    w_smoothness=settings.W_SMOOTHNESS,
                    w_left_right=settings.W_LEFT_RIGHT)
            loss.backward()

            delta_grad = delta.grad.detach()
            delta.data = torch.clamp(delta + (alpha*torch.sign(delta_grad)), -epsilon, epsilon)
            delta = delta.detach()

            advImage = torch.clamp(advImage + delta, 0, 1)
            optimizer.zero_grad()

        pert_img = advImage
        ######################################

        pert_depth_all = model.forward(pert_img, camera)
        pert_depth = np.squeeze(pert_depth_all[2].detach().cpu().numpy())
        pert_depth_rescaled = cv2.resize(pert_depth, dsize=(1242, 375), interpolation=cv2.INTER_LINEAR)

    #     if i % 50 == 0:
    #         plt.imshow(np.transpose(np.squeeze(image0.detach().cpu().numpy()), (1, 2, 0)))
    #         plt.show()
    #         plt.imshow(np.squeeze(pred_depth[0].detach().cpu().numpy()))
    #         plt.show()

    #         pertubation = np.squeeze(delta.detach().cpu().numpy())
    #         normPert = (pertubation - np.amin(pertubation)) / (np.amax(pertubation) - np.amin(pertubation))

    #         plt.imshow(np.transpose(normPert, (1, 2, 0)))
    #         plt.show()
    #         plt.imshow(np.transpose(np.squeeze(pert_img.detach().cpu().numpy()), (1, 2, 0)))
    #         plt.show()
    #         plt.imshow(np.squeeze(pert_depth_all[0].detach().cpu().numpy()))
    #         plt.show()


        #Compare images
        #Convert nan depths to max, as likey is to be inf depth due to 0 disparity.
        pred_depth_orig = np.nan_to_num(pred_depth_orig, nan=max_depth)
        pred_depth_orig[pred_depth_orig < min_depth] = min_depth
        pred_depth_orig[pred_depth_orig > max_depth] = max_depth

        pert_depth = np.nan_to_num(pert_depth, nan=max_depth)
        pert_depth[pert_depth < min_depth] = min_depth
        pert_depth[pert_depth > max_depth] = max_depth

        abs_rel[0][i], sq_rel[0][i], rms[0][i], log_rms[0][i], a1[0][i], a2[0][i], a3[0][i] = compute_errors(pred_depth_orig, pert_depth)



        gt_depth = gt_depths[i]

        #Compute for only values inbetween 0 and 80 as the ground truth velodyne data is very sparse.
        mask = np.logical_and(gt_depth > min_depth, gt_depth < max_depth)

        pert_depth_rescaled = np.nan_to_num(pert_depth_rescaled, nan=max_depth)
        pert_depth_rescaled[pert_depth_rescaled < min_depth] = min_depth
        pert_depth_rescaled[pert_depth_rescaled > max_depth] = max_depth

        abs_rel[1][i], sq_rel[1][i], rms[1][i], log_rms[1][i], a1[1][i], a2[1][i], a3[1][i] = compute_errors(gt_depth[mask], pert_depth_rescaled[mask])

        i+=1
    print(abs_rel.mean(axis=1))
    print(sq_rel.mean(axis=1))
    print(rms.mean(axis=1))
    print(log_rms.mean(axis=1))